# Single Agent Demo

This notebook demonstrates how to interact with a **single agent** using the .NET Agent Framework backend API.

## What You'll Learn
- How to get available agents
- How to send messages to a single agent
- How to handle responses
- Different response formats (user-friendly vs detailed)

## Prerequisites
Make sure the backend API is running on `http://localhost:8000`

## Setup - Install Required Packages

In [ ]:
// Install required NuGet packages
#r "nuget:System.Net.Http.Json"
#r "nuget:System.Text.Json"

using System;
using System.Net.Http;
using System.Net.Http.Json;
using System.Text.Json;
using System.Text.Json.Serialization;
using System.Linq;

Console.WriteLine("✅ Packages installed and namespaces imported successfully!");

## Configure API Connection

In [ ]:
// API Configuration
var apiBaseUrl = "http://localhost:8000";
var httpClient = new HttpClient { BaseAddress = new Uri(apiBaseUrl) };
httpClient.Timeout = TimeSpan.FromMinutes(2);

Console.WriteLine($"✅ Connected to API: {apiBaseUrl}");

## Step 1: Get Available Agents

## Test API Connection

Let's first verify that the backend API is running and accessible.

In [ ]:
// Quick test to verify API is accessible
try
{
    var testResponse = await httpClient.GetAsync("/agents");
    Console.WriteLine($"✅ API Status: {testResponse.StatusCode}");
    
    if (testResponse.IsSuccessStatusCode)
    {
        Console.WriteLine("✅ Backend API is running and accessible!");
    }
    else
    {
        Console.WriteLine($"⚠️  API returned status: {testResponse.StatusCode}");
        Console.WriteLine("Make sure the backend is running with: dotnet run");
    }
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Cannot connect to API: {ex.Message}");
    Console.WriteLine("Please start the backend API first:");
    Console.WriteLine("   cd Backend\\dotnet");
    Console.WriteLine("   dotnet run");
}

In [ ]:
// Get all available agents
var agentsResponse = await httpClient.GetFromJsonAsync<JsonElement>("/agents");

Console.WriteLine("📋 Available Agents:");
Console.WriteLine("===================");

var agents = agentsResponse.GetProperty("agents");
foreach (var agent in agents.EnumerateArray())
{
    var name = agent.GetProperty("name").GetString();
    var type = agent.GetProperty("type").GetString();
    var provider = agent.GetProperty("provider").GetString();
    
    Console.WriteLine($"\n🤖 Agent: {name}");
    Console.WriteLine($"   Type: {type}");
    Console.WriteLine($"   Provider: {provider}");
    
    if (agent.TryGetProperty("capabilities", out var caps))
    {
        var capabilities = string.Join(", ", caps.EnumerateArray().Select(c => c.GetString()));
        if (!string.IsNullOrEmpty(capabilities))
        {
            Console.WriteLine($"   Capabilities: {capabilities}");
        }
    }
}

var totalAgents = agentsResponse.GetProperty("total").GetInt32();
Console.WriteLine($"\n✅ Total available agents: {totalAgents}");

## Step 2: Send a Simple Message to a Single Agent (User-Friendly Format)

In [ ]:
// Prepare chat request for a single agent
var chatRequest = new
{
    message = "What is artificial intelligence and how is it used in healthcare?",
    agents = new[] { "generic_agent" },  // Single agent
    session_id = (string?)null,  // New session
    format = "user_friendly"  // Default format
};

Console.WriteLine("📤 Sending message to single agent...");
Console.WriteLine($"Message: {chatRequest.message}");
Console.WriteLine($"Agent: {chatRequest.agents[0]}\n");

// Send the request
var response = await httpClient.PostAsJsonAsync("/chat", chatRequest);
response.EnsureSuccessStatusCode();

var result = await response.Content.ReadFromJsonAsync<JsonElement>();

// Display response
Console.WriteLine("\n💬 Response:");
Console.WriteLine("=============");
Console.WriteLine($"Agent: {result.GetProperty("agent").GetString()}");
Console.WriteLine($"Session ID: {result.GetProperty("session_id").GetString()}");
Console.WriteLine($"\nContent:\n{result.GetProperty("content").GetString()}");

// Display metadata
if (result.TryGetProperty("metadata", out var metadata))
{
    Console.WriteLine($"\n📊 Metadata:");
    Console.WriteLine($"   Format: {metadata.GetProperty("response_type").GetString()}");
    Console.WriteLine($"   Agent Count: {metadata.GetProperty("agent_count").GetInt32()}");
    Console.WriteLine($"   Total Turns: {metadata.GetProperty("total_turns").GetInt32()}");
}

## Step 3: Send a Message with Detailed Format

The detailed format provides more information including individual agent responses, turn information, and metadata.

In [ ]:
// Request with detailed format
var detailedRequest = new
{
    message = "Explain machine learning in simple terms.",
    agents = new[] { "generic_agent" },
    format = "detailed",  // Detailed format for more information
    max_turns = 2
};

Console.WriteLine("📤 Sending message with DETAILED format...");
Console.WriteLine($"Message: {detailedRequest.message}\n");

var detailedResponse = await httpClient.PostAsJsonAsync("/chat", detailedRequest);
detailedResponse.EnsureSuccessStatusCode();

var detailedResult = await detailedResponse.Content.ReadFromJsonAsync<JsonElement>();

// Display detailed response
Console.WriteLine("\n💬 Detailed Response:");
Console.WriteLine("=====================");
Console.WriteLine($"Conversation ID: {detailedResult.GetProperty("conversation_id").GetString()}");
Console.WriteLine($"Total Turns: {detailedResult.GetProperty("total_turns").GetInt32()}");

// Show individual responses
var responses = detailedResult.GetProperty("responses");
Console.WriteLine($"\n📝 Individual Agent Responses ({responses.GetArrayLength()}):");

foreach (var resp in responses.EnumerateArray())
{
    Console.WriteLine($"\n  🤖 Agent: {resp.GetProperty("agent").GetString()}");
    Console.WriteLine($"     Turn: {resp.GetProperty("metadata").GetProperty("turn").GetInt32()}");
    Console.WriteLine($"     Content: {resp.GetProperty("content").GetString()}");
    Console.WriteLine($"     Timestamp: {resp.GetProperty("metadata").GetProperty("timestamp").GetString()}");
}

// Summary
if (detailedResult.TryGetProperty("summary", out var summary) && summary.ValueKind != JsonValueKind.Null)
{
    Console.WriteLine($"\n📋 Summary:\n{summary.GetString()}");
}

// Metadata
var meta = detailedResult.GetProperty("metadata");
Console.WriteLine($"\n📊 Metadata:");
Console.WriteLine($"   Response Type: {meta.GetProperty("response_type").GetString()}");
Console.WriteLine($"   Group Chat Type: {meta.GetProperty("group_chat_type").GetString()}");
Console.WriteLine($"   Agent Count: {meta.GetProperty("agent_count").GetInt32()}");

## Step 4: Auto-select Agent (No Agent Specified)

If you don't specify an agent, the system will automatically select one for you.

In [ ]:
// Request without specifying an agent
var autoRequest = new
{
    message = "Tell me a fun fact about space.",
    // No agents specified - will be auto-selected
    format = "user_friendly"
};

Console.WriteLine("📤 Sending message with AUTO-SELECTED agent...");
Console.WriteLine($"Message: {autoRequest.message}\n");

var autoResponse = await httpClient.PostAsJsonAsync("/chat", autoRequest);
autoResponse.EnsureSuccessStatusCode();

var autoResult = await autoResponse.Content.ReadFromJsonAsync<JsonElement>();

Console.WriteLine("\n💬 Response:");
Console.WriteLine("=============");
Console.WriteLine($"Auto-selected Agent: {autoResult.GetProperty("agent").GetString()}");
Console.WriteLine($"\nContent:\n{autoResult.GetProperty("content").GetString()}");

## Step 5: Working with Different Agent Types

Try sending messages to different types of agents to see how they respond differently.

In [ ]:
// Try different agents based on what's available
var agentTypes = new[] { "generic_agent", "people_lookup", "knowledge_finder" };

foreach (var agentType in agentTypes)
{
    try
    {
        Console.WriteLine($"\n🔄 Testing agent: {agentType}");
        Console.WriteLine("=".PadRight(50, '='));
        
        var testRequest = new
        {
            message = "Hello! What can you help me with?",
            agents = new[] { agentType },
            format = "user_friendly"
        };
        
        var testResponse = await httpClient.PostAsJsonAsync("/chat", testRequest);
        
        if (testResponse.IsSuccessStatusCode)
        {
            var testResult = await testResponse.Content.ReadFromJsonAsync<JsonElement>();
            var content = testResult.GetProperty("content").GetString();
            var truncatedContent = content.Length > 200 ? content.Substring(0, 200) + "..." : content;
            
            Console.WriteLine($"✅ Agent responded: {truncatedContent}");
        }
        else
        {
            Console.WriteLine($"⚠️  Agent not available or error occurred");
        }
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error testing {agentType}: {ex.Message}");
    }
}

## Summary

In this notebook, you learned:
- ✅ How to retrieve available agents from the API
- ✅ How to send messages to a single agent
- ✅ The difference between user-friendly and detailed response formats
- ✅ How to use auto-agent selection
- ✅ How to work with different agent types

## Next Steps
- Try the **Multiple Agent Demo** notebook to see how agents collaborate
- Explore the **Content Safety Demo** for content moderation features
- Check out the **Memory Demo** for session management and conversation history